In [2]:
import findspark
findspark.init()

Creating a Spark Session

In [3]:
from pyspark.sql import SparkSession

In [4]:
spark = SparkSession.builder.config("spark.serializer", "org.apache.spark.serializer.KryoSerializer").getOrCreate()

Creating Dataframes

In [5]:
import pyspark.sql.functions as F
from pyspark.sql.types import *

In [6]:
schema = StructType([ \
    StructField("user_id", StringType(), True), \
    StructField("country", StringType(), True), \
    StructField("browser", StringType(), True), \
    StructField("OS", StringType(), True), \
    StructField("age", IntegerType(), True) \
    ])

In [7]:
data2 = [("A203",'India',"Chrome","WIN", 33),
         ("A201",'China',"Safari","MacOS",35),
         ("A205",'UK',"Mozilla","Linux",25)]

In [8]:
df = spark.createDataFrame(data=data2,schema=schema)

In [9]:
df.printSchema()

root
 |-- user_id: string (nullable = true)
 |-- country: string (nullable = true)
 |-- browser: string (nullable = true)
 |-- OS: string (nullable = true)
 |-- age: integer (nullable = true)



In [10]:
df.show()

+-------+-------+-------+-----+---+
|user_id|country|browser|   OS|age|
+-------+-------+-------+-----+---+
|   A203|  India| Chrome|  WIN| 33|
|   A201|  China| Safari|MacOS| 35|
|   A205|     UK|Mozilla|Linux| 25|
+-------+-------+-------+-----+---+



Null Values

In Spark, we can deal with null values by either replacing them with some specific value or dropping the rows/columns containing null values

In [11]:
df_na = spark.createDataFrame([("A203",None,"Chrome","WIN",33),
                               ("A201",'China',None,"MacOS",35),
                               ("A205",'UK',"Mozilla", "Linux",25)],schema=schema)

In [12]:
df_na.show()

+-------+-------+-------+-----+---+
|user_id|country|browser|   OS|age|
+-------+-------+-------+-----+---+
|   A203|   null| Chrome|  WIN| 33|
|   A201|  China|   null|MacOS| 35|
|   A205|     UK|Mozilla|Linux| 25|
+-------+-------+-------+-----+---+



In [13]:
df_na.fillna('0').show()

+-------+-------+-------+-----+---+
|user_id|country|browser|   OS|age|
+-------+-------+-------+-----+---+
|   A203|      0| Chrome|  WIN| 33|
|   A201|  China|      0|MacOS| 35|
|   A205|     UK|Mozilla|Linux| 25|
+-------+-------+-------+-----+---+



In [14]:
df_na.fillna({'country':'USA', 'browser':'Safari'}).show()

+-------+-------+-------+-----+---+
|user_id|country|browser|   OS|age|
+-------+-------+-------+-----+---+
|   A203|    USA| Chrome|  WIN| 33|
|   A201|  China| Safari|MacOS| 35|
|   A205|     UK|Mozilla|Linux| 25|
+-------+-------+-------+-----+---+



In [15]:
df_na.na.drop().show()

+-------+-------+-------+-----+---+
|user_id|country|browser|   OS|age|
+-------+-------+-------+-----+---+
|   A205|     UK|Mozilla|Linux| 25|
+-------+-------+-------+-----+---+



In [16]:
df_na.na.drop(subset="country").show()

+-------+-------+-------+-----+---+
|user_id|country|browser|   OS|age|
+-------+-------+-------+-----+---+
|   A201|  China|   null|MacOS| 35|
|   A205|     UK|Mozilla|Linux| 25|
+-------+-------+-------+-----+---+



In [17]:
df_na.replace("Chrome", "Google Chrome").show()

+-------+-------+-------------+-----+---+
|user_id|country|      browser|   OS|age|
+-------+-------+-------------+-----+---+
|   A203|   null|Google Chrome|  WIN| 33|
|   A201|  China|         null|MacOS| 35|
|   A205|     UK|      Mozilla|Linux| 25|
+-------+-------+-------------+-----+---+



In [18]:
df_na.drop("user_id").show()

+-------+-------+-----+---+
|country|browser|   OS|age|
+-------+-------+-----+---+
|   null| Chrome|  WIN| 33|
|  China|   null|MacOS| 35|
|     UK|Mozilla|Linux| 25|
+-------+-------+-----+---+



In [19]:
df=spark.read.options(delimiter=',', inferSchema='True', header='True').csv("data/Invistico_Airline.csv")

In [20]:
df.count()

129880

In [21]:
len(df.columns)

23

In [22]:
df.printSchema()

root
 |-- satisfaction: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Customer Type: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Type of Travel: string (nullable = true)
 |-- Class: string (nullable = true)
 |-- Flight Distance: integer (nullable = true)
 |-- Seat comfort: integer (nullable = true)
 |-- Departure/Arrival time convenient: integer (nullable = true)
 |-- Food and drink: integer (nullable = true)
 |-- Gate location: integer (nullable = true)
 |-- Inflight wifi service: integer (nullable = true)
 |-- Inflight entertainment: integer (nullable = true)
 |-- Online support: integer (nullable = true)
 |-- Ease of Online booking: integer (nullable = true)
 |-- On-board service: integer (nullable = true)
 |-- Leg room service: integer (nullable = true)
 |-- Baggage handling: integer (nullable = true)
 |-- Checkin service: integer (nullable = true)
 |-- Cleanliness: integer (nullable = true)
 |-- Online boarding: integer (nullable = true)

In [23]:
df.show(3)

+------------+------+--------------+---+---------------+--------+---------------+------------+---------------------------------+--------------+-------------+---------------------+----------------------+--------------+----------------------+----------------+----------------+----------------+---------------+-----------+---------------+--------------------------+------------------------+
|satisfaction|Gender| Customer Type|Age| Type of Travel|   Class|Flight Distance|Seat comfort|Departure/Arrival time convenient|Food and drink|Gate location|Inflight wifi service|Inflight entertainment|Online support|Ease of Online booking|On-board service|Leg room service|Baggage handling|Checkin service|Cleanliness|Online boarding|Departure Delay in Minutes|Arrival Delay in Minutes|
+------------+------+--------------+---+---------------+--------+---------------+------------+---------------------------------+--------------+-------------+---------------------+----------------------+--------------+-------

In [24]:
df.summary().show()

+-------+------------+------+-----------------+------------------+---------------+--------+-----------------+-----------------+---------------------------------+------------------+------------------+---------------------+----------------------+------------------+----------------------+------------------+------------------+-----------------+------------------+------------------+------------------+--------------------------+------------------------+
|summary|satisfaction|Gender|    Customer Type|               Age| Type of Travel|   Class|  Flight Distance|     Seat comfort|Departure/Arrival time convenient|    Food and drink|     Gate location|Inflight wifi service|Inflight entertainment|    Online support|Ease of Online booking|  On-board service|  Leg room service| Baggage handling|   Checkin service|       Cleanliness|   Online boarding|Departure Delay in Minutes|Arrival Delay in Minutes|
+-------+------------+------+-----------------+------------------+---------------+--------+-----

Subset of a Dataframe
A subset of a dataframe can be created, based on multiple conditions in which we either select a few rows, columns, or data with certain filters in place.
• Select
• Filter
• Where

Select

In [27]:
df.select(['satisfaction','Class', 'Gender']).show()

+------------+--------+------+
|satisfaction|   Class|Gender|
+------------+--------+------+
|   satisfied|     Eco|Female|
|   satisfied|Business|  Male|
|   satisfied|     Eco|Female|
|   satisfied|     Eco|Female|
|   satisfied|     Eco|Female|
|   satisfied|     Eco|  Male|
|   satisfied|     Eco|Female|
|   satisfied|     Eco|  Male|
|   satisfied|Business|Female|
|   satisfied|     Eco|  Male|
|   satisfied|     Eco|Female|
|   satisfied|     Eco|Female|
|   satisfied|     Eco|  Male|
|   satisfied|     Eco|  Male|
|   satisfied|     Eco|Female|
|   satisfied|     Eco|  Male|
|   satisfied|     Eco|Female|
|   satisfied|Business|Female|
|   satisfied|     Eco|Female|
|   satisfied|     Eco|Female|
+------------+--------+------+
only showing top 20 rows



In [28]:
df.filter(df['Age'] > 60).count()

10054

In [29]:
df.filter(df['Age'] > 60).show()

+------------+------+--------------+---+---------------+--------+---------------+------------+---------------------------------+--------------+-------------+---------------------+----------------------+--------------+----------------------+----------------+----------------+----------------+---------------+-----------+---------------+--------------------------+------------------------+
|satisfaction|Gender| Customer Type|Age| Type of Travel|   Class|Flight Distance|Seat comfort|Departure/Arrival time convenient|Food and drink|Gate location|Inflight wifi service|Inflight entertainment|Online support|Ease of Online booking|On-board service|Leg room service|Baggage handling|Checkin service|Cleanliness|Online boarding|Departure Delay in Minutes|Arrival Delay in Minutes|
+------------+------+--------------+---+---------------+--------+---------------+------------+---------------------------------+--------------+-------------+---------------------+----------------------+--------------+-------

Filter
We can also apply more than one filter on the dataframe, by including more conditions.

In [31]:
df.filter(df['Age'] > 50).filter(df['Gender'] == 'Male').count()

16721

Where

In [33]:
df.where((df['Age'] > 55) & (df['Class'] == 'Eco')).count()

10252

Aggregations
Any kind of aggregation can be broken simply into three stages, in the following order:
• Split
• Apply
• Combine

The first step is to split the data, based on a column or group of columns, followed by performing the operation on those small individual groups (count, max, avg, etc.). Once the results are in for each set of groups, the last step is to combine all these results.

In [34]:
 df.groupBy('Gender').count().show()

+------+-----+
|Gender|count|
+------+-----+
|Female|65899|
|  Male|63981|
+------+-----+



In [36]:
for col in df.columns:
    if col !='Age':
        print(f" Aggregation for {col}")
df.groupBy(col).count().orderBy('count',ascending= False).show(truncate=False)

 Aggregation for satisfaction
 Aggregation for Gender
 Aggregation for Customer Type
 Aggregation for Type of Travel
 Aggregation for Class
 Aggregation for Flight Distance
 Aggregation for Seat comfort
 Aggregation for Departure/Arrival time convenient
 Aggregation for Food and drink
 Aggregation for Gate location
 Aggregation for Inflight wifi service
 Aggregation for Inflight entertainment
 Aggregation for Online support
 Aggregation for Ease of Online booking
 Aggregation for On-board service
 Aggregation for Leg room service
 Aggregation for Baggage handling
 Aggregation for Checkin service
 Aggregation for Cleanliness
 Aggregation for Online boarding
 Aggregation for Departure Delay in Minutes
 Aggregation for Arrival Delay in Minutes
+------------------------+-----+
|Arrival Delay in Minutes|count|
+------------------------+-----+
|0                       |72753|
|1                       |2747 |
|2                       |2587 |
|3                       |2442 |
|4                

We can have different kinds of operations on groups of records, such as:
• Mean
• Max
• Min
• Sum

In [37]:
df.groupBy('Class').agg(F.mean('Age')).show()

+--------+-----------------+
|   Class|         avg(Age)|
+--------+-----------------+
|Eco Plus|38.72893422590585|
|Business|41.61727799227799|
|     Eco|37.20686343446123|
+--------+-----------------+



In [39]:
df.groupBy('Class').agg(F.max('Age')).show()

+--------+--------+
|   Class|max(Age)|
+--------+--------+
|Eco Plus|      85|
|Business|      85|
|     Eco|      85|
+--------+--------+



In [40]:
df.groupBy('Customer Type').agg(F.min('Age')).show()

+-----------------+--------+
|    Customer Type|min(Age)|
+-----------------+--------+
|   Loyal Customer|       7|
|disloyal Customer|       7|
+-----------------+--------+



In [41]:
df.groupBy('Customer Type').agg(F.sum('Flight Distance')).show()

+-----------------+--------------------+
|    Customer Type|sum(Flight Distance)|
+-----------------+--------------------+
|   Loyal Customer|           209250457|
|disloyal Customer|            48094951|
+-----------------+--------------------+



In [42]:
df.sort("Age", ascending=False).show()

+------------+------+-----------------+---+---------------+--------+---------------+------------+---------------------------------+--------------+-------------+---------------------+----------------------+--------------+----------------------+----------------+----------------+----------------+---------------+-----------+---------------+--------------------------+------------------------+
|satisfaction|Gender|    Customer Type|Age| Type of Travel|   Class|Flight Distance|Seat comfort|Departure/Arrival time convenient|Food and drink|Gate location|Inflight wifi service|Inflight entertainment|Online support|Ease of Online booking|On-board service|Leg room service|Baggage handling|Checkin service|Cleanliness|Online boarding|Departure Delay in Minutes|Arrival Delay in Minutes|
+------------+------+-----------------+---+---------------+--------+---------------+------------+---------------------------------+--------------+-------------+---------------------+----------------------+-------------

In [44]:
df.groupBy('Customer Type').agg(F.avg('Age').alias('avg_age')).orderBy('avg_age',ascending=False).show(truncate=False)

+-----------------+-----------------+
|Customer Type    |avg_age          |
+-----------------+-----------------+
|Loyal Customer   |41.46200754005655|
|disloyal Customer|30.35256518082422|
+-----------------+-----------------+



In [45]:
df.groupBy('Class').agg(F.max('Flight Distance').alias('max_fl_distance')).orderBy('max_fl_distance',ascending=False).show()

+--------+---------------+
|   Class|max_fl_distance|
+--------+---------------+
|Business|           6951|
|     Eco|           6924|
|Eco Plus|           6889|
+--------+---------------+



Collect
Collect list provides all the values in the original order of occurrence (they can be reversed as well), and collect set provides only the unique values

In [48]:
df.groupby("Customer Type", "Class").agg(F.collect_set("Online boarding")).show()

+-----------------+--------+----------------------------+
|    Customer Type|   Class|collect_set(Online boarding)|
+-----------------+--------+----------------------------+
|   Loyal Customer|Business|          [0, 1, 2, 3, 4, 5]|
|   Loyal Customer|     Eco|          [0, 1, 2, 3, 4, 5]|
|disloyal Customer|Eco Plus|             [1, 2, 3, 4, 5]|
|   Loyal Customer|Eco Plus|          [0, 1, 2, 3, 4, 5]|
|disloyal Customer|     Eco|             [1, 2, 3, 4, 5]|
|disloyal Customer|Business|             [1, 2, 3, 4, 5]|
+-----------------+--------+----------------------------+



In [49]:
df.groupby("Customer Type", "Class").agg(F.collect_list("Online boarding")).show()

+-----------------+--------+-----------------------------+
|    Customer Type|   Class|collect_list(Online boarding)|
+-----------------+--------+-----------------------------+
|   Loyal Customer|Business|         [2, 4, 3, 1, 2, 3...|
|   Loyal Customer|     Eco|         [2, 2, 3, 5, 2, 3...|
|disloyal Customer|Eco Plus|         [3, 5, 4, 2, 5, 4...|
|   Loyal Customer|Eco Plus|         [2, 5, 5, 1, 2, 5...|
|disloyal Customer|     Eco|         [4, 2, 2, 2, 1, 4...|
|disloyal Customer|Business|         [1, 4, 5, 5, 5, 2...|
+-----------------+--------+-----------------------------+



The need to create a new column with a constant value can be very common. Therefore, we can do that in PySpark, using the 'lit' function.

In [50]:
df=df.withColumn('constant',F.lit('resolved'))

In [51]:
 df.select('Customer Type', 'Gender', 'constant').show()

+--------------+------+--------+
| Customer Type|Gender|constant|
+--------------+------+--------+
|Loyal Customer|Female|resolved|
|Loyal Customer|  Male|resolved|
|Loyal Customer|Female|resolved|
|Loyal Customer|Female|resolved|
|Loyal Customer|Female|resolved|
|Loyal Customer|  Male|resolved|
|Loyal Customer|Female|resolved|
|Loyal Customer|  Male|resolved|
|Loyal Customer|Female|resolved|
|Loyal Customer|  Male|resolved|
|Loyal Customer|Female|resolved|
|Loyal Customer|Female|resolved|
|Loyal Customer|  Male|resolved|
|Loyal Customer|  Male|resolved|
|Loyal Customer|Female|resolved|
|Loyal Customer|  Male|resolved|
|Loyal Customer|Female|resolved|
|Loyal Customer|Female|resolved|
|Loyal Customer|Female|resolved|
|Loyal Customer|Female|resolved|
+--------------+------+--------+
only showing top 20 rows

